In [ ]:
# ! pip install google-cloud-vision

In [ ]:
import cv2
import imutils
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import requests
import time
from base64 import b64encode
from IPython.display import Image
from pylab import rcParams
import io

from google.cloud import vision
import proto

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="your credentials in json"

In [ ]:
os.chdir('/content/WARD8/')

In [ ]:
def generate_code(file_name):
  image_uri = f"{file_name}.jpg"
  output_file = f"{file_name}.xlsx"

  client = vision.ImageAnnotatorClient()
  image = vision.Image()
  image.source.image_uri = image_uri

  client = vision.ImageAnnotatorClient()

  # The name of the image file to annotate
  file_name = os.path.abspath(image_uri)

  # Loads the image into memory
  with io.open(file_name, 'rb') as image_file:
      content = image_file.read()

  image = vision.Image(content=content)
  response = client.text_detection(image=image)
  texts = proto.Message.to_json(response)
  mydict = json.loads(texts)

  data = mydict['fullTextAnnotation']['text'].split('\n')[8:]

  name = []
  father_name = []
  # husband_name = []
  household_count = []
  age = []
  gender = []

  for i in data[:-4]:
    i = i.replace("|", "")
    i = i.replace("Photo is", "")
    i = i.replace("Available", "")
    if ":" in i:
      if i.count(":") < 2:
        if "निर्वाचक का नाम" in i.split(":")[0].strip():
          name.append(i.split(":")[1].strip())
        elif "पिता का नाम" in i.split(":")[0].strip() :
          father_name.append(i.split(":")[1].strip())   
          # husband_name.append("NA")  
        elif "माता का नाम" in i.split(":")[0].strip() :
          father_name.append(i.split(":")[1].strip())   
          # husband_name.append("NA")  
        elif "पति का नाम" in i.split(":")[0].strip():
          father_name.append(i.split(":")[1].strip())  
          # father_name.append("NA")  
        elif "अन्य का नाम" in i.split(":")[0].strip():
          father_name.append(i.split(":")[1].strip())  
          # father_name.append("NA")  

    
        elif "गृह संख्या" in i.split(":")[0].strip():
          household_count.append(i.split(":")[1].strip())   
        elif "उम्र" in i.split(":")[0].strip():
          age.append(i.split(":")[1].strip())  
        elif "लिंग" in i.split(":")[0].strip():
          gender.append(i.split(":")[1].strip())             
        # print("")
      
  print(len(name))
  print(len(father_name)) 
  # print(len(husband_name)) 
  print(len(household_count))
  print(len(age))
  print(len(gender))

  new_df = pd.DataFrame({'Name': name,
                       "father/mother/husband": father_name,
                      #  "husband_name": husband_name,
                       "household_count": household_count,
                       "age": age,
                       "gender": gender})
  
  new_df['Ward No.'] = 8
  output_file = f"{file_name}.xlsx"
  new_df.to_excel(output_file, index=False)

  return new_df

In [ ]:
for filenames in os.listdir():
  if "WARD" in filenames:
    fname = filenames.split(".")[0]
    print(fname)
    new_df = generate_code(fname)